In [6]:
#!import AutomatedAnalysis.dib

  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> C:\Users\musharm\source\repos\performance3\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.94


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [7]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";

## Display Benchmark Regressions

In [8]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Out of 30 benchmarks, 6 have regressed.
   - __ConnectionClose__ has regressed by 24.77% for Average of Latency 50th
   - __JsonHttpSys__ has regressed by 17.54% for Average of Latency 50th
   - __Json__ has regressed by 12.09% for Average of Latency 50th
   - __Stage1Pgo__ has regressed by 5.31% for Average of Latency 50th
   - __ConnectionClose__ has regressed by -5.24% for Average of Max heap size


### Per-benchmark behavior

|          Benchmark Name | Average of Max heap size / 28c | Average of Max heap size / 28c_con1 | Average of Max heap size /  Comparison % | Average of Latency 50th / 28c | Average of Latency 50th / 28c_con1 | Average of Latency 50th /  Comparison % |
| ----------------------: | -----------------------------: | ----------------------------------: | ---------------------------------------: | ----------------------------: | ---------------------------------: | --------------------------------------: |
|         ConnectionClose |                          7.575 |                               7.178 |                                   -5.241 |                         1.100 |                              1.373 |                                  24.773 |
|             JsonHttpSys |                         34.516 |                              48.246 |                                   39.780 |                         0.527 |                              0.620 |                                  17.536 |
|                    Json |                          4.110 |                               4.102 |                                   -0.186 |                         0.228 |                              0.255 |                                  12.088 |
| PlaintextPlatformInline |                          7.521 |                               5.877 |                                  -21.858 |                         1.038 |                              1.045 |                                   0.723 |
|       PlaintextPlatform |                          5.158 |                               3.234 |                                  -37.314 |                         0.862 |                              0.853 |                                  -1.159 |
|               Stage1Pgo |                         30.603 |                              38.501 |                                   25.807 |                         0.283 |                              0.298 |                                   5.310 |

## Display Iteration Anomalies

In [13]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 0.5 },
    { Metrics.I.Latency50thMS, 3 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ) //, benchmarkFilter: new Filter("Stage1Pgo"),
,volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides, benchmarksToDisplay: 10); 

Error: (32,76): error CS1739: The best overload for 'DisplayIterationData' does not have a parameter named 'benchmarksToDisplay'

## Display GC Charts

In [10]:
ChartGCData(dm, metrics: ML( Metrics.G.MedianThroughputCostPercent, Metrics.G.NumHeaps, Metrics.G.TotalSOHStableSize ), 
 //iterationFilter: new IntFilter(1), 
 benchmarkFilter: new Filter("PlaintextMvc", "Stage1Pgo"),
 xMetric: Metrics.X.StartRelativeMSec)

Too many units: %, #, bytes


<!DOCTYPE html>

<!DOCTYPE html>

index value 0 <!DOCTYPE html>